<a href="https://colab.research.google.com/github/ieg-dhr/NLP-Course4Humanities_2024/blob/main/Named_Entity_Recognition_ImpressoAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualizing Named Entities with Impresso API

The models and corresponding API implementation used in this notebook were developed by Emanuale Boros [ORCID](https://orcid.org/0000-0001-6299-9452) from the Impresso Project (https://impresso-project.ch/). The notebook was created by Sarah Oberbichler for teaching purposes [ORCID](https://orcid.org/0000-0002-1031-2759).

##About this Notebook

This notebook enables the extraction of named entities from any dataset using the Impresso NER and Named Entity Linking (NEL) API, providing precise recognition of fine-grained named entities in historical texts across German, French, and English. Furthermore, it demonstrates how to visualize location entities by generating geographic coordinates for the extracted locations and plotting them on an interactive map, enhancing both analysis and presentation.

**Acknowledgements:**

For further experimentation, you can directly access the experimental API at [Impresso Annotation](https://impresso-annotation.epfl.ch/).


##Importing a Dataset

In [ ]:
!git clone https://github.com/ieg-dhr/NLP-Course4Humanities_2024.git

In [ ]:
import pandas as pd

articles_df = pd.read_excel('/content/NLP-Course4Humanities_2024/datasets/earthquake_articles.xlsx')

articles_df = articles_df[:20]
articles_df.head()

## Call the Impresso NER and NEL API
We only use the NER part for this notebook.

In [ ]:
!pip install --upgrade --force-reinstall impresso
from impresso import version
print(version)

In [ ]:
from impresso import connect
impresso_session = connect()

##Extracting Locations from the NER Output - An Example

We can extract specific entities in order to use them for further analysis or visualization. Please not that when using extracted named entities for further analysis, they need to be controlled and verified by a human reader since the model most likely has made some mistakes.

In [ ]:
import pandas as pd
import time
from typing import Optional

def process_text_with_ner(text: str, max_retries: int = 3, delay: int = 2) -> Optional[dict]:
    """
    Process text with NER, including retry logic and error handling
    """
    if pd.isna(text):
        return None

    # Add consistent delay before each request
    time.sleep(1)  # Wait 1 second between requests

    for attempt in range(max_retries):
        try:
            result = impresso_session.tools.ner(text=text)
            return result
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
                delay *= 2
            continue
    return None

def process_dataframe(df: pd.DataFrame, text_column: str = 'extracted_article_clean') -> pd.DataFrame:
    results = []

    for idx, row in df.iterrows():
        print(f"Processing row {idx + 1}/{len(df)}")
        try:
            text = row[text_column]
            result = process_text_with_ner(text)
            results.append(result)
        except KeyError:
            print(f"Column '{text_column}' not found. Available columns: {df.columns}")
            raise
        except Exception as e:
            print(f"Error processing row {idx + 1}: {str(e)}")
            results.append(None)

    df['ner_results'] = results
    return df

# Process the DataFrame using extracted_article_clean column
articles_df = process_dataframe(articles_df)


In [ ]:
def extract_locations(df):
    places = []
    for index, row in df.iterrows():
        ner_container = row['ner_results']
        if ner_container is not None:
            try:
                # Get DataFrame from NerContainer
                ner_df = ner_container.df
                # Filter for locations
                loc_rows = ner_df[ner_df['type'] == 'loc']
                # Get the surface forms directly
                loc_names = loc_rows['surfaceForm'].tolist()
                places.append(', '.join(loc_names) if loc_names else '')
            except Exception as e:
                print(f"Error processing row {index}: {str(e)}")
                places.append('')
        else:
            places.append('')

    return places

# Apply the function
articles_df['places'] = extract_locations(articles_df)

# Show results
print("\nResults:")
print(articles_df[['extracted_article_clean', 'places']].head())

In [ ]:
# export as excel file
articles_df.to_excel('earthquake_articles_with_ner.xlsx', index=False)

## Visualization Example - Creating a Map with Place Names

We first use the geopy library to process geographic locations and add their corresponding coordinates (latitude and longitude) to a pandas DataFrame. It includes a GeocodingService class that interfaces with the Nominatim geocoding API, implementing rate-limiting, retries with exponential backoff, and error handling to ensure robust geocoding.

We further use the folium library to create an interactive map with markers for locations provided in a pandas DataFrame. Finally, the map is created and displayed, providing a visual representation of the geographic data.

In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import pandas as pd
import time
from typing import List, Tuple, Optional
import random

class GeocodingService:
    def __init__(self, user_agent: str = None, timeout: int = 10, rate_limit: float = 1.1):
        """
        Initialize the geocoding service with proper configuration.

        Args:
            user_agent: Custom user agent string (default: generated)
            timeout: Timeout for requests in seconds
            rate_limit: Time to wait between requests in seconds
        """
        if user_agent is None:
            user_agent = f"python_geocoding_script_{random.randint(1000, 9999)}"

        self.geolocator = Nominatim(
            user_agent=user_agent,
            timeout=timeout
        )
        self.rate_limit = rate_limit
        self.last_request = 0

    def _rate_limit_wait(self):
        """Implement rate limiting between requests"""
        current_time = time.time()
        time_since_last = current_time - self.last_request
        if time_since_last < self.rate_limit:
            time.sleep(self.rate_limit - time_since_last)
        self.last_request = time.time()

    def geocode_location(self, location: str, max_retries: int = 3) -> Optional[Tuple[float, float]]:
        """
        Geocode a single location with retries.

        Args:
            location: Location string to geocode
            max_retries: Maximum number of retry attempts

        Returns:
            Tuple of (latitude, longitude) or None if geocoding fails
        """
        for attempt in range(max_retries):
            try:
                self._rate_limit_wait()
                location_data = self.geolocator.geocode(location)
                if location_data:
                    return (location_data.latitude, location_data.longitude)
                return None
            except (GeocoderTimedOut, GeocoderServiceError) as e:
                if attempt == max_retries - 1:
                    print(f"Failed to geocode '{location}' after {max_retries} attempts: {e}")
                    return None
                time.sleep(2 ** attempt)  # Exponential backoff
            except Exception as e:
                print(f"Error geocoding '{location}': {e}")
                return None
        return None

    def process_locations(self, locations: str) -> List[Optional[Tuple[float, float]]]:
        """
        Process a comma-separated string of locations.

        Args:
            locations: Comma-separated string of location names

        Returns:
            List of coordinate tuples or None for failed geocoding
        """
        if pd.isna(locations) or not locations:
            return []

        location_list = [loc.strip() for loc in locations.split(',')]
        return [self.geocode_location(loc) for loc in location_list]

def geolocate_places(df: pd.DataFrame,
                    places_column: str = 'places',
                    user_agent: str = None) -> pd.DataFrame:
    """
    Add coordinates to a DataFrame based on location names.

    Args:
        df: Input DataFrame
        places_column: Name of the column containing comma-separated location strings
        user_agent: Custom user agent string

    Returns:
        DataFrame with added 'coordinates' column
    """
    geocoder = GeocodingService(user_agent=user_agent)

    # Create a copy to avoid modifying the original DataFrame
    result_df = df.copy()

    # Process locations
    result_df['coordinates'] = result_df[places_column].apply(geocoder.process_locations)

    return result_df

# Main execution
if __name__ == "__main__":
    # Assuming articles_df is your DataFrame with a 'places' column
    # Apply geocoding to the articles DataFrame
    articles_df_with_coords = geolocate_places(
        articles_df,
        places_column='places',
        user_agent='article_geocoding_service_v1.0'
    )

    # Update the original DataFrame with the new coordinates
    articles_df['coordinates'] = articles_df_with_coords['coordinates']

    # Display the results
    print("\nSample of geocoded locations:")
    print(articles_df[['places', 'coordinates']].head())

    # Optional: Display some statistics
    total_locations = len(articles_df)
    successful_geocodes = articles_df['coordinates'].apply(lambda x: len([c for c in x if c is not None])).sum()
    failed_geocodes = articles_df['coordinates'].apply(lambda x: len([c for c in x if c is None])).sum()

    print(f"\nGeocoding Statistics:")
    print(f"Total locations processed: {total_locations}")
    print(f"Successfully geocoded: {successful_geocodes}")
    print(f"Failed to geocode: {failed_geocodes}")

In [ ]:
import folium
from folium import plugins
import pandas as pd
from typing import List, Tuple, Optional
from IPython.display import display

def create_location_map(df: pd.DataFrame,
                       coordinates_col: str = 'coordinates',
                       places_col: str = 'places',
                       title_col: Optional[str] = None) -> folium.Map:
    """
    Create an interactive map with markers for all locations in the DataFrame.

    Args:
        df: DataFrame containing coordinates and place names
        coordinates_col: Name of column containing coordinates
        places_col: Name of column containing place names
        title_col: Optional column name for additional marker information

    Returns:
        folium.Map object with all locations marked
    """
    # Initialize the map
    m = folium.Map(location=[0, 0], zoom_start=2)

    # Create a MarkerCluster
    marker_cluster = plugins.MarkerCluster().add_to(m)

    # Keep track of all valid coordinates for setting bounds
    all_coords = []

    # Process each row in the DataFrame
    for idx, row in df.iterrows():
        coordinates = row[coordinates_col]
        places = row[places_col].split(',') if pd.notna(row[places_col]) else []
        title = row[title_col] if title_col and pd.notna(row[title_col]) else None

        # Skip if no coordinates
        if not coordinates:
            continue

        # Add markers for each location
        for i, (coord, place) in enumerate(zip(coordinates, places)):
            if coord is not None:  # Skip None coordinates
                lat, lon = coord
                place_name = place.strip()

                # Create popup content
                popup_content = f"<b>{place_name}</b>"
                if title:
                    popup_content += f"<br>{title}"

                # Add marker
                folium.Marker(
                    location=[lat, lon],
                    popup=popup_content,
                    tooltip=place_name
                ).add_to(marker_cluster)

                all_coords.append([lat, lon])

    # If we have coordinates, fit the map bounds to include all points
    if all_coords:
        m.fit_bounds(all_coords)

    return m

# Create and display the map
map_obj = create_location_map(articles_df)
display(map_obj)